In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr
from PIL import Image
from transformers import AutoTokenizer,ViTFeatureExtractor,VisionEncoderDecoderModel

In [ ]:
device='cpu'
encode_="nlpconnect/vit-gpt2-image-captioning"
decode_="nlpconnect/vit-gpt2-image-captioning"
model_="nlpconnect/vit-gpt2-image-captioning"

tokenizer=AutoTokenizer.from_pretrained(decode_)
feature_extractor=ViTFeatureExtractor.from_pretrained(encode_)
model=VisionEncoderDecoderModel.from_pretrained(model_).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
def pred(image,max_length=64,num_beams=4,num_outputs=3):
  image=image.convert('RGB')
  image=feature_extractor(image, return_tensors='pt').pixel_values.to(device)
  caption_ids=model.generate(image, max_length=max_length,num_beams=num_beams,num_return_sequences=num_outputs)
  captions=[tokenizer.decode(ids)for ids in caption_ids]
  return captions

def captioning_fn(image):
  pil_image=Image.fromarray(image.astype('uint8'),'RGB')
  captions=pred(pil_image)
  return ('Captions:',captions)

def set_example_image(example: list)->dict:
  return gr.Image.update(value=example[o])

css='''
h1{
  text-align: center;
}
h3{
  text-align:center;
}
img{
  max_width=800px;
  max_height=600px;
}
img{
  max_width=1000px;
  max_height=600px;
}'''

demo=gr.Blocks(css=css)
with demo:
  gr.Markdown('''<h1 id='title'> Image Caption Generator</h1>''')
  with gr.Column():
    input=gr.inputs.Image(type='pil',label='Upload Image',optional=True)
    output=gr.outputs.Textbox(type='text',label='Caption')
  btn=gr.Button("Generate Captions")
  btn.click(fn=pred,inputs=input,outputs=output)

demo.launch()

<ipython-input-6-a680c6e8f67e>:36: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  input=gr.inputs.Image(type='pil',label='Upload Image',optional=True)
<ipython-input-6-a680c6e8f67e>:36: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  input=gr.inputs.Image(type='pil',label='Upload Image',optional=True)
<ipython-input-6-a680c6e8f67e>:37: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output=gr.outputs.Textbox(type='text',label='Caption')


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>